Only keep (not ablate) the query activations (query vector, the output of query weights times inputs) of certain positions. Keep all the key activations; the query positions that were kept will automatically attend to relevant key positions by matrix multiplication.

# Setup
(No need to change anything)

In [1]:
# Janky code to do different setup when run in a Colab notebook vs VSCode
DEBUG_MODE = False
try:
    import google.colab
    IN_COLAB = True
    print("Running as a Colab notebook")
    %pip install git+https://github.com/neelnanda-io/TransformerLens.git
    # Install another version of node that makes PySvelte work way faster
    !curl -fsSL https://deb.nodesource.com/setup_16.x | sudo -E bash -; sudo apt-get install -y nodejs
    %pip install git+https://github.com/neelnanda-io/PySvelte.git
except:
    IN_COLAB = False
    print("Running as a Jupyter notebook - intended for development only!")
    from IPython import get_ipython

    ipython = get_ipython()
    # Code to automatically update the HookedTransformer code as its edited without restarting the kernel
    ipython.magic("load_ext autoreload")
    ipython.magic("autoreload 2")

Running as a Colab notebook
  Cloning https://github.com/neelnanda-io/TransformerLens.git to /tmp/pip-req-build-7k3h9jqq
  Running command git clone --filter=blob:none --quiet https://github.com/neelnanda-io/TransformerLens.git /tmp/pip-req-build-7k3h9jqq
  Resolved https://github.com/neelnanda-io/TransformerLens.git to commit bf10751f2e293d05d5116d36539ad34d91a025ae
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 739.7/739.7 kB 31.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 45.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 72.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.0 MB/s eta 0:00:00
    

In [2]:
# Plotly needs a different renderer for VSCode/Notebooks vs Colab argh
import plotly.io as pio

if IN_COLAB or not DEBUG_MODE:
    # Thanks to annoying rendering issues, Plotly graphics will either show up in colab OR Vscode depending on the renderer - this is bad for developing demos! Thus creating a debug mode.
    pio.renderers.default = "colab"
else:
    pio.renderers.default = "png"

In [3]:
# Import stuff
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import einops
from fancy_einsum import einsum
import tqdm.notebook as tqdm
import random
from pathlib import Path
import plotly.express as px
from torch.utils.data import DataLoader

from jaxtyping import Float, Int
from typing import List, Union, Optional
from functools import partial
import copy

import itertools
from transformers import AutoModelForCausalLM, AutoConfig, AutoTokenizer
import dataclasses
import datasets
from IPython.display import HTML

In [4]:
# import pysvelte

import transformer_lens
import transformer_lens.utils as utils
from transformer_lens.hook_points import (
    HookedRootModule,
    HookPoint,
)  # Hooking utilities
from transformer_lens import HookedTransformer, HookedTransformerConfig, FactoredMatrix, ActivationCache

We turn automatic differentiation off, to save GPU memory, as this notebook focuses on model inference not model training.

In [5]:
torch.set_grad_enabled(False)

Plotting helper functions:

In [6]:
def imshow(tensor, renderer=None, **kwargs):
    px.imshow(utils.to_numpy(tensor), color_continuous_midpoint=0.0, color_continuous_scale="RdBu", **kwargs).show(renderer)

def line(tensor, renderer=None, **kwargs):
    px.line(y=utils.to_numpy(tensor), **kwargs).show(renderer)

def scatter(x, y, xaxis="", yaxis="", caxis="", renderer=None, **kwargs):
    x = utils.to_numpy(x)
    y = utils.to_numpy(y)
    px.scatter(y=y, x=x, labels={"x":xaxis, "y":yaxis, "color":caxis}, **kwargs).show(renderer)

## Load Model

Decide which model to use (eg. gpt2-small vs -medium)

In [7]:
model = HookedTransformer.from_pretrained(
    "gpt2-small",
    center_unembed=True,
    center_writing_weights=True,
    fold_ln=True,
    refactor_factored_attn_matrices=True,
)

Loaded pretrained model gpt2-small into HookedTransformer


## Import functions from repo

In [8]:
!git clone https://github.com/callummcdougall/ARENA_2.0.git

Cloning into 'ARENA_2.0'...
remote: Enumerating objects: 9100, done.
remote: Counting objects: 100% (1814/1814), done.
remote: Compressing objects: 100% (288/288), done.
remote: Total 9100 (delta 1609), reused 1602 (delta 1523), pack-reused 7286
Receiving objects: 100% (9100/9100), 155.60 MiB | 22.60 MiB/s, done.
Resolving deltas: 100% (5502/5502), done.


In [9]:
cd ARENA_2.0/chapter1_transformers/exercises/part3_indirect_object_identification

/content/ARENA_2.0/chapter1_transformers/exercises/part3_indirect_object_identification


In [10]:
import ioi_circuit_extraction as ioi_circuit_extraction

# Generate dataset with multiple prompts

In [11]:
class Dataset:
    def __init__(self, prompts, tokenizer, S1_is_first=False):
        self.prompts = prompts
        self.tokenizer = tokenizer
        self.N = len(prompts)
        self.max_len = max(
            [
                len(self.tokenizer(prompt["text"]).input_ids)
                for prompt in self.prompts
            ]
        )
        # all_ids = [prompt["TEMPLATE_IDX"] for prompt in self.ioi_prompts]
        all_ids = [0 for prompt in self.prompts] # only 1 template
        all_ids_ar = np.array(all_ids)
        self.groups = []
        for id in list(set(all_ids)):
            self.groups.append(np.where(all_ids_ar == id)[0])

        texts = [ prompt["text"] for prompt in self.prompts ]
        self.toks = torch.Tensor(self.tokenizer(texts, padding=True).input_ids).type(
            torch.int
        )
        self.io_tokenIDs = [
            self.tokenizer.encode(" " + prompt["S5"])[0] for prompt in self.prompts
        ]
        self.s_tokenIDs = [
            self.tokenizer.encode(" " + prompt["S4"])[0] for prompt in self.prompts
        ]

        # word_idx: for every prompt, find the token index of each target token and "end"
        # word_idx is a tensor with an element for each prompt. The element is the targ token's ind at that prompt
        self.word_idx = {}
        for targ in [key for key in self.prompts[0].keys() if (key != 'text' and key != 'S5')]:
            targ_lst = []
            for prompt in self.prompts:
                input_text = prompt["text"]
                tokens = model.tokenizer.tokenize(input_text)
                if S1_is_first and targ == "S1":  # only use this if first token doesn't have space Ġ in front
                    target_token = prompt[targ]
                else:
                    target_token = "Ġ" + prompt[targ]
                target_index = tokens.index(target_token)
                targ_lst.append(target_index)
            self.word_idx[targ] = torch.tensor(targ_lst)

        targ_lst = []
        for prompt in self.prompts:
            input_text = prompt["text"]
            tokens = self.tokenizer.tokenize(input_text)
            end_token_index = len(tokens) - 1
            targ_lst.append(end_token_index)
        self.word_idx["end"] = torch.tensor(targ_lst)

    def __len__(self):
        return self.N

Repalce io_tokens with correct answer (next, which is '5') and s_tokens with incorrect (current, which repeats)

In [12]:
def generate_prompts_list(x ,y):
    prompts_list = []
    for i in range(x, y):
        prompt_dict = {
            'S1': str(i),
            'S2': str(i+1),
            'S3': str(i+2),
            'S4': str(i+3),
            'S5': str(i+4),
            'text': f"{i} {i+1} {i+2} {i+3}"
        }
        prompts_list.append(prompt_dict)
    return prompts_list

prompts_list = generate_prompts_list(1, 11)
dataset = Dataset(prompts_list, model.tokenizer, S1_is_first=True)

In [13]:
def generate_prompts_list_corr(x ,y):
    prompts_list = []
    for i in range(x, y):
        prompt_dict = {
            'S1': str(i),
            'S2': str(i+1),
            'S3': str(i+2),
            'S4': str(i+2),
            'S5': str(i+3),
            'text': f"{i} {i+1} {i+2} {i+2}"
        }
        prompts_list.append(prompt_dict)
    return prompts_list

prompts_list_2 = generate_prompts_list_corr(1, 11)
dataset_2 = Dataset(prompts_list_2, model.tokenizer, S1_is_first=True)

Logit diff is correct - incorr token. Here, correct is S5, and incorr is S4.

Because of this, it's possible to have logit diffs HIGHER than the "full circuit" because the correct token will still be at first place, but the logit scores assigned will just be bigger (perhaps incorrect is scored even lower in the non-full circuit with a higher logit diff score)?

# Ablation Expm Functions

In [14]:
from torch import Tensor

def logits_to_ave_logit_diff_2(logits: Float[Tensor, "batch seq d_vocab"], dataset: Dataset, per_prompt=False):
    '''
    Returns logit difference between the correct and incorrect answer.

    If per_prompt=True, return the array of differences rather than the average.
    '''

    # Only the final logits are relevant for the answer
    # Get the logits corresponding to the indirect object / subject tokens respectively
    io_logits: Float[Tensor, "batch"] = logits[range(logits.size(0)), dataset.word_idx["end"], dataset.io_tokenIDs]
    s_logits: Float[Tensor, "batch"] = logits[range(logits.size(0)), dataset.word_idx["end"], dataset.s_tokenIDs]
    # Find logit difference
    answer_logit_diff = io_logits - s_logits
    return answer_logit_diff if per_prompt else answer_logit_diff.mean()

In [15]:
def mean_ablate_by_lst(CIRCUIT, SEQ_POS_TO_KEEP, model, print_output=True):
    model.reset_hooks(including_permanent=True)  #must do this after running with mean ablation hook

    ioi_logits_original, ioi_cache = model.run_with_cache(dataset.toks)

    model = ioi_circuit_extraction.add_mean_ablation_hook(model, means_dataset=dataset_2, circuit=CIRCUIT, seq_pos_to_keep=SEQ_POS_TO_KEEP)
    ioi_logits_minimal = model(dataset.toks)

    orig_score = logits_to_ave_logit_diff_2(ioi_logits_original, dataset)
    new_score = logits_to_ave_logit_diff_2(ioi_logits_minimal, dataset)
    if print_output:
        # print(f"Average logit difference (IOI dataset, using entire model): {orig_score:.4f}")
        # print(f"Average logit difference (IOI dataset, only using circuit): {new_score:.4f}")
        print(f"Average logit difference (circuit / full) %: {100 * new_score / orig_score:.4f}")
    # return new_score
    return 100 * new_score / orig_score

We can also prevent redundant computation of the full circuit score by storing it and just passing it in to the function.

# Ablate the model and compare with original

Try among words

https://colab.research.google.com/drive/1CHRn-AMko9RNrl1bqiCwB7DS-rz1CoBP#scrollTo=KZiVdGTC6QlP&line=2&uniqifier=1

In [ ]:
circuit = [(0, 1), (0, 3), (0, 5), (0, 7), (0, 9), (0, 10), (1, 5), (2, 2), (2, 9), (3, 0), (3, 3), (3, 7), (4, 4), (5, 5), (6, 1), (6, 6), (6, 9), (6, 10), (7, 10), (7, 11), (8, 8), (9, 1), (10, 7)]

In [18]:
# CIRCUIT = {
#     "number mover": lst,
#     "number mover 4": lst,
#     "number mover 3": lst,
#     "number mover 2": lst,
#     "number mover 1": lst,
# }

# SEQ_POS_TO_KEEP = {
#     "number mover": "end",
#     "number mover 4": "S4",
#     "number mover 3": "S3",
#     "number mover 2": "S2",
#     "number mover 1": "S1",
# }

earlyHeads = [(0, 1), (0, 3), (0, 5), (0, 7), (0, 9), (0, 10), (1, 5), (2, 2), (2, 9), (3, 0), (3, 3), (3, 7), (4, 4), (5, 5), (6, 1), (6, 6), (6, 9), (6, 10)]

CIRCUIT = {
    "number mover": [(7, 10), (7, 11), (8, 8), (9, 1), (10, 7)],
    "number detectors 4": earlyHeads,
    "number detectors 3": earlyHeads,
    "number detectors 2": earlyHeads,
    "number detectors 1": earlyHeads,
}

SEQ_POS_TO_KEEP = {
    "number mover": "end",
    "number detectors 4": "S4",
    "number detectors 3": "S3",
    "number detectors 2": "S2",
    "number detectors 1": "S1",
}

In [19]:
mean_ablate_by_lst(CIRCUIT, SEQ_POS_TO_KEEP, model, print_output=False).item()

-0.012001964263617992

Check that it's not an error in the code, but it actually gets this when number movers only attend from the end pos query

In [20]:
# all number movers attend from all pos, but earlyHeads don't attend from end

CIRCUIT = {
    "number mover": [(7, 10), (7, 11), (8, 8), (9, 1), (10, 7)],
    "number mover 4": [(7, 10), (7, 11), (8, 8), (9, 1), (10, 7)],
    "number mover 3": [(7, 10), (7, 11), (8, 8), (9, 1), (10, 7)],
    "number mover 2": [(7, 10), (7, 11), (8, 8), (9, 1), (10, 7)],
    "number mover 1": [(7, 10), (7, 11), (8, 8), (9, 1), (10, 7)],
    "number detectors 4": earlyHeads,
    "number detectors 3": earlyHeads,
    "number detectors 2": earlyHeads,
    "number detectors 1": earlyHeads,
}

SEQ_POS_TO_KEEP = {
    "number mover": "end",
    "number mover 4": "S4",
    "number mover 3": "S3",
    "number mover 2": "S2",
    "number mover 1": "S1",
    "number detectors 4": "S4",
    "number detectors 3": "S3",
    "number detectors 2": "S2",
    "number detectors 1": "S1",
}

mean_ablate_by_lst(CIRCUIT, SEQ_POS_TO_KEEP, model, print_output=False).item()

-0.021190611645579338

In [22]:
# all heads attend from all pos. this should match the prev code's impl: https://colab.research.google.com/drive/1CHRn-AMko9RNrl1bqiCwB7DS-rz1CoBP#scrollTo=hiGhLPyD6TiB&line=1&uniqifier=1

CIRCUIT = {
    "number mover": [(7, 10), (7, 11), (8, 8), (9, 1), (10, 7)],
    "number mover 4": [(7, 10), (7, 11), (8, 8), (9, 1), (10, 7)],
    "number mover 3": [(7, 10), (7, 11), (8, 8), (9, 1), (10, 7)],
    "number mover 2": [(7, 10), (7, 11), (8, 8), (9, 1), (10, 7)],
    "number mover 1": [(7, 10), (7, 11), (8, 8), (9, 1), (10, 7)],
    "number detectors": earlyHeads,
    "number detectors 4": earlyHeads,
    "number detectors 3": earlyHeads,
    "number detectors 2": earlyHeads,
    "number detectors 1": earlyHeads,
}

SEQ_POS_TO_KEEP = {
    "number mover": "end",
    "number mover 4": "S4",
    "number mover 3": "S3",
    "number mover 2": "S2",
    "number mover 1": "S1",
    "number detectors": "end",
    "number detectors 4": "S4",
    "number detectors 3": "S3",
    "number detectors 2": "S2",
    "number detectors 1": "S1",
}

mean_ablate_by_lst(CIRCUIT, SEQ_POS_TO_KEEP, model, print_output=False).item()

75.5345458984375

It's approximate. This means the "number detectors" must STILL attend from EVERYWHERE. Not just query from a pos.

Now just